In [155]:
import pandas
import json
from dateutil import parser
import re

In [156]:
def filter_non_operator_words(text):
    # Keep only words that start with $
    operators = re.findall(r'\$\w+',text)
    operators.append('')
    operator_filtered = text
    for operator in operators:
        operator_filtered = operator_filtered.replace(operator,"")
    non_operators = re.findall(r'[a-z0-9A-Z]+',operator_filtered)
    for non_op in non_operators:
        text = text.replace(non_op,"")
    return text#' '.join(filtered_words)

In [157]:
filter_non_operator_words("{username: {$regex: ^{}}")

'{: {$regex: ^{}}'

In [158]:
with open('../Dataset/queryLogs.json','r') as log_file:
    log_data = json.load(log_file)
print(log_data)

with open("../No-SQL_Gen/No-SqlDataset.json") as query_file:
    query_data = json.load(query_file)

[{'t': {'$date': '2024-04-04T20:37:10.296+05:30'}, 's': 'I', 'c': 'COMMAND', 'id': 51803, 'ctx': 'conn10', 'msg': 'Slow query', 'attr': {'type': 'command', 'ns': 'test_database.test_collection', 'command': {'find': 'test_collection', 'filter': {'user': '{}', 'password': '{}'}, 'lsid': {'id': {'$uuid': 'e5e23d7e-5367-4829-a104-ca369c49d4dc'}}, '$db': 'test_database'}, 'planSummary': 'EOF', 'planningTimeMicros': 83, 'keysExamined': 0, 'docsExamined': 0, 'nBatches': 1, 'cursorExhausted': True, 'numYields': 0, 'nreturned': 0, 'queryFramework': 'classic', 'reslen': 118, 'locks': {'FeatureCompatibilityVersion': {'acquireCount': {'r': 1}}, 'Global': {'acquireCount': {'r': 1}}}, 'storage': {}, 'cpuNanos': 176371, 'remote': '127.0.0.1:47038', 'protocol': 'op_msg', 'durationMillis': 0}}, {'t': {'$date': '2024-04-04T20:37:10.297+05:30'}, 's': 'I', 'c': 'COMMAND', 'id': 51803, 'ctx': 'conn10', 'msg': 'Slow query', 'attr': {'type': 'command', 'ns': 'test_database.test_collection', 'command': {'find

In [159]:
# Loading other Data

# MongoDB Attribute Columns
with open("../Dataset/MongoDB_attributes_list.json") as MongoDB_attr_data:
    mongo_db_attribute_data = json.load(MongoDB_attr_data)

mongo_db_attribute_columns = mongo_db_attribute_data["mongo_db_attr"]
command_columns = mongo_db_attribute_data["command_columns"]
query_operators = mongo_db_attribute_data["query_operators"]

In [160]:
log_data_frame = pandas.DataFrame(log_data)
log_data_frame.head()

,t,s,c,id,ctx,msg,attr
0,{'$date': '2024-04-04T20:37:10.296+05:30'},I,COMMAND,51803,conn10,Slow query,"{'type': 'command', 'ns': 'test_database.test_..."
1,{'$date': '2024-04-04T20:37:10.297+05:30'},I,COMMAND,51803,conn10,Slow query,"{'type': 'command', 'ns': 'test_database.test_..."
2,{'$date': '2024-04-04T20:37:10.297+05:30'},I,COMMAND,51803,conn10,Slow query,"{'type': 'command', 'ns': 'test_database.test_..."
3,{'$date': '2024-04-04T20:37:10.298+05:30'},I,COMMAND,51803,conn10,Slow query,"{'type': 'command', 'ns': 'test_database.test_..."
4,{'$date': '2024-04-04T20:37:10.298+05:30'},I,COMMAND,51803,conn10,Slow query,"{'type': 'command', 'ns': 'test_database.test_..."


In [161]:
query_data_frame = pandas.DataFrame(query_data)
query_data_frame['text'] = query_data_frame['text'].astype(str)
query_data_frame['query_length_raw'] = query_data_frame['text'].str.len()
query_data_frame['keywords_only'] = query_data_frame['text'].str.replace("\"","")
query_data_frame['keywords_only'] = query_data_frame['keywords_only'].apply(filter_non_operator_words)
query_data_frame['query_length_keywords_only'] = query_data_frame['keywords_only'].str.len()
query_data_frame.head()

,text,label,query_length_raw,keywords_only,query_length_keywords_only
0,"{""user"": ""{}"", ""password"": ""{}""}",0,32,"{: {}, : {}}",12
1,"{""username"": {""$regex"": ""^{}""}}",1,31,{: {$regex: ^{}}},17
2,"{""username"": {""$ne"": ""{}""}}",1,27,{: {$ne: {}}},13
3,"{""username"": {""$gt"": ""{}""}}",1,27,{: {$gt: {}}},13
4,"{""user"": ""hacker"", ""password"": {""$ne"": """"}}",1,43,"{: , : {$ne: }}",15


In [162]:
# Cleaning Data

#Convert T (Timestamp) to Timestamps
log_data_frame_cleaned = pandas.DataFrame()
log_data_frame_cleaned['t'] = log_data_frame['t'].apply(lambda x: parser.parse(x['$date']).timestamp())

#Extract Attr Columns in Nested attr Column
for col in mongo_db_attribute_columns:
    log_data_frame_cleaned[col] = log_data_frame['attr'].apply(lambda x: x.get(col, None))

#Extract Command Columns in Nested command Column
for col in command_columns:
    log_data_frame_cleaned[col] = log_data_frame_cleaned['command'].apply(lambda x: x.get(col, None) if isinstance(x, dict) else None)

#Convert Other Object Columns to str for hashing (For later Operations)
log_data_frame_cleaned[log_data_frame_cleaned.select_dtypes(include=['object']).columns] = log_data_frame_cleaned.select_dtypes(include=['object']).astype(str)

# Find which Particular Query Operator Exists in filter column
for operatorType in query_operators['all_operators']:
    log_data_frame_cleaned[operatorType] = log_data_frame_cleaned['filter'].apply(lambda x:1 if operatorType in x else 0)

# Find which Classes of Query Operators Exists in filter Column
for operatorClass in query_operators:
    log_data_frame_cleaned[operatorClass] = log_data_frame_cleaned['filter'].apply(lambda x:1 if any(s in x for s in operatorClass) else 0)
    
# Create Joining Column
log_data_frame_cleaned["text"] = log_data_frame_cleaned['filter']
log_data_frame_cleaned["text"] = log_data_frame_cleaned["text"].str.replace('\'', '\"')

#Drop extra columns
log_data_frame_cleaned = log_data_frame_cleaned.drop(['command'],axis=1)


log_data_frame_cleaned.head()

,t,type,ns,planSummary,planningTimeMicros,keysExamined,docsExamined,nBatches,cursorExhausted,numYields,...,selector_array,selector_bitwise,projection,misc,selector,standard_logical,all_operators,null_operand,regex_null_operand,text
0,1.712243e+09,command,test_database.test_collection,EOF,83.0,0.0,0.0,1.0,True,0,...,1,1,1,1,1,1,1,1,1,"{""user"": ""{}"", ""password"": ""{}""}"
1,1.712243e+09,command,test_database.test_collection,EOF,71.0,0.0,0.0,1.0,True,0,...,1,1,1,1,1,1,1,1,1,"{""username"": {""$regex"": ""^{}""}}"
2,1.712243e+09,command,test_database.test_collection,EOF,42.0,0.0,0.0,1.0,True,0,...,1,1,1,1,1,1,1,1,1,"{""username"": {""$ne"": ""{}""}}"
3,1.712243e+09,command,test_database.test_collection,EOF,46.0,0.0,0.0,1.0,True,0,...,1,1,1,1,1,1,1,1,1,"{""username"": {""$gt"": ""{}""}}"
4,1.712243e+09,command,test_database.test_collection,EOF,83.0,0.0,0.0,1.0,True,0,...,1,1,1,1,1,1,1,1,1,"{""user"": ""hacker"", ""password"": {""$ne"": """"}}"


In [163]:
log_data_frame_cleaned.dtypes

t                     float64
type                   object
ns                     object
planSummary            object
planningTimeMicros    float64
                       ...   
standard_logical        int64
all_operators           int64
null_operand            int64
regex_null_operand      int64
text                   object
Length: 79, dtype: object

In [164]:
query_data_frame.dtypes

text                          object
label                          int64
query_length_raw               int64
keywords_only                 object
query_length_keywords_only     int64
dtype: object

In [165]:
log_data_frame_cleaned['ns'].nunique()

3

In [166]:
# Remove Columns with Constant Value
constant_columns = [col for col in log_data_frame_cleaned.columns if log_data_frame_cleaned[col].nunique() == 1]
log_data_frame_optimized = log_data_frame_cleaned.drop(columns=constant_columns)
log_data_frame_optimized.head()

,t,ns,planSummary,planningTimeMicros,cursorExhausted,queryFramework,reslen,locks,cpuNanos,durationMillis,...,selector_array,selector_bitwise,projection,misc,selector,standard_logical,all_operators,null_operand,regex_null_operand,text
0,1.712243e+09,test_database.test_collection,EOF,83.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,176371,0,...,1,1,1,1,1,1,1,1,1,"{""user"": ""{}"", ""password"": ""{}""}"
1,1.712243e+09,test_database.test_collection,EOF,71.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,112781,0,...,1,1,1,1,1,1,1,1,1,"{""username"": {""$regex"": ""^{}""}}"
2,1.712243e+09,test_database.test_collection,EOF,42.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,71093,0,...,1,1,1,1,1,1,1,1,1,"{""username"": {""$ne"": ""{}""}}"
3,1.712243e+09,test_database.test_collection,EOF,46.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,81173,0,...,1,1,1,1,1,1,1,1,1,"{""username"": {""$gt"": ""{}""}}"
4,1.712243e+09,test_database.test_collection,EOF,83.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,141175,0,...,1,1,1,1,1,1,1,1,1,"{""user"": ""hacker"", ""password"": {""$ne"": """"}}"


In [167]:
# Removed Columns

constant_columns

['type',
 'keysExamined',
 'docsExamined',
 'nBatches',
 'numYields',
 'nreturned',
 'storage',
 'remote',
 'protocol',
 '$db',
 '$and',
 '$nor',
 '$gte',
 '$lt',
 'lte',
 '$exists',
 '$expr',
 '$jsonSchema',
 '$text',
 '$all',
 '$bitsAllClear',
 '$bitsAllSet',
 '$bitsAnyClear',
 '$bitsAnySet',
 'meta',
 '$slice',
 '$rand',
 '$natural',
 '!=',
 '||',
 '!']

In [168]:
log_data_frame_optimized['text'].head()

0               {"user": "{}", "password": "{}"}
1                {"username": {"$regex": "^{}"}}
2                    {"username": {"$ne": "{}"}}
3                    {"username": {"$gt": "{}"}}
4    {"user": "hacker", "password": {"$ne": ""}}
Name: text, dtype: object

In [169]:
# Join dataframes

log_data_frame_cleaned_merged_output = pandas.merge(log_data_frame_cleaned,query_data_frame,on='text')
log_data_frame_cleaned_merged_output.head()
log_data_frame_optimized_merged_output = pandas.merge(log_data_frame_optimized,query_data_frame,on='text')
log_data_frame_optimized_merged_output.head()

,t,ns,planSummary,planningTimeMicros,cursorExhausted,queryFramework,reslen,locks,cpuNanos,durationMillis,...,selector,standard_logical,all_operators,null_operand,regex_null_operand,text,label,query_length_raw,keywords_only,query_length_keywords_only
0,1.712243e+09,test_database.test_collection,EOF,83.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,176371,0,...,1,1,1,1,1,"{""user"": ""{}"", ""password"": ""{}""}",0,32,"{: {}, : {}}",12
1,1.712243e+09,test_database.test_collection,EOF,83.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,176371,0,...,1,1,1,1,1,"{""user"": ""{}"", ""password"": ""{}""}",0,32,"{: {}, : {}}",12
2,1.712243e+09,test_database.test_collection,EOF,71.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,112781,0,...,1,1,1,1,1,"{""username"": {""$regex"": ""^{}""}}",1,31,{: {$regex: ^{}}},17
3,1.712243e+09,test_database.test_collection,EOF,71.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,112781,0,...,1,1,1,1,1,"{""username"": {""$regex"": ""^{}""}}",1,31,{: {$regex: ^{}}},17
4,1.712243e+09,test_database.test_collection,EOF,42.0,True,classic,118,{'FeatureCompatibilityVersion': {'acquireCount...,71093,0,...,1,1,1,1,1,"{""username"": {""$ne"": ""{}""}}",1,27,{: {$ne: {}}},13


In [170]:
# Redo Removal of constant column since join has removed rows
constant_columns_joined = [col for col in log_data_frame_optimized_merged_output.columns if log_data_frame_optimized_merged_output[col].nunique() == 1]
log_data_frame_optimized_merged_output = log_data_frame_optimized_merged_output.drop(columns=constant_columns_joined)
log_data_frame_optimized_merged_output.head()

,t,planSummary,planningTimeMicros,cursorExhausted,queryFramework,reslen,cpuNanos,filter,$eq,$gt,...,selector,standard_logical,all_operators,null_operand,regex_null_operand,text,label,query_length_raw,keywords_only,query_length_keywords_only
0,1.712243e+09,EOF,83.0,True,classic,118,176371,"{'user': '{}', 'password': '{}'}",0,0,...,1,1,1,1,1,"{""user"": ""{}"", ""password"": ""{}""}",0,32,"{: {}, : {}}",12
1,1.712243e+09,EOF,83.0,True,classic,118,176371,"{'user': '{}', 'password': '{}'}",0,0,...,1,1,1,1,1,"{""user"": ""{}"", ""password"": ""{}""}",0,32,"{: {}, : {}}",12
2,1.712243e+09,EOF,71.0,True,classic,118,112781,{'username': {'$regex': '^{}'}},0,0,...,1,1,1,1,1,"{""username"": {""$regex"": ""^{}""}}",1,31,{: {$regex: ^{}}},17
3,1.712243e+09,EOF,71.0,True,classic,118,112781,{'username': {'$regex': '^{}'}},0,0,...,1,1,1,1,1,"{""username"": {""$regex"": ""^{}""}}",1,31,{: {$regex: ^{}}},17
4,1.712243e+09,EOF,42.0,True,classic,118,71093,{'username': {'$ne': '{}'}},0,0,...,1,1,1,1,1,"{""username"": {""$ne"": ""{}""}}",1,27,{: {$ne: {}}},13


In [171]:
# Removed Columns in joined version

constant_columns_joined

['ns',
 'locks',
 'durationMillis',
 'find',
 'lsid',
 '$not',
 '$or',
 '==',
 '===',
 '&&']

In [172]:
# Save All

log_data_frame_cleaned.to_csv("../processed_datasets/log_data_cleaned.csv",encoding='utf-8')
log_data_frame_optimized.to_csv("../processed_datasets/log_data_cleaned_optimized.csv",encoding='utf-8')
log_data_frame_cleaned_merged_output.to_csv("../processed_datasets/log_data_cleaned_merged.csv",encoding='utf-8')
log_data_frame_optimized_merged_output.to_csv("../processed_datasets/log_data_optimized_merged.csv",encoding='utf-8')